In [ ]:
# 690e25b4-8c5e-4a10-a32e-523da88a4c99 is my demo tenant
$TenantId = '690e25b4-8c5e-4a10-a32e-523da88a4c99' #"<TENANT_ID>"
$SubscriptionId = 'b9334351-cec8-405d-8358-51846fa2a3ab'

. "..\src\functions.ps1"

try{
    $null = Set-AzContext -TenantId $TenantId -SubscriptionId $SubscriptionId -ErrorAction Stop
}
catch{
    $null = Connect-AzAccount -TenantId $TenantId
}

Get-AzSubscription -TenantId $TenantId | Select-Object -Property Name, Id

# Subnets should have an NSG associated

Below shows subnets with no NSG associated. Delegated subnets are less important to protect using NSGs, but should still be so if at all possible.

In [ ]:
$Query = @"
resources
| where type == "microsoft.network/virtualnetworks"
| mvexpand subnets = properties.subnets
| extend Delegated = iif(array_length(subnets.properties.delegations) == 0, "no", "yes")
| where isempty(subnets.properties.networkSecurityGroup)
| project ["VNET Name"] = name, ["Subnet Name"] = subnets.name, Delegated, id = subnets.id
"@

Search-AzGraph -Query $Query -UseTenantScope | Select-Object -ExpandProperty Data | Select-Object -ExcludeProperty ResourceId | Format-Table -AutoSize

# NSG Deny All Rule

All subnets should have a deny all rule ("deny by default, permit by exception" approach). This can be implemented with a priority of 4096 (last rule evaluated always), which counters some of the default rules that ex. allows VNET to VNET (traffic tagged as VNET).

Below outputs the NSGs without such a rule.

In [ ]:
<#
powershell:
find NSGs with a "deny all" rule. as azgraph does not support antijoins we will have to find the NSGs that does NOT have such a rule
#>
$Query = @"
resources
| where type == "microsoft.network/networksecuritygroups"
| extend p = todynamic(properties.securityRules)
| mvexpand p
| where p.properties.access == "Deny" and p.properties.direction == "Inbound" and p.properties.destinationPortRange == '*' and p.properties.sourceAddressPrefix == '*'
| extend nsg_id = substring(p.id, 0, strlen(p.id) - strlen(strcat('/securityRules/',p.name)))
| join (
	resources
	| where type == "microsoft.network/networksecuritygroups"
	| project name, nsg_id = id
) on nsg_id
| project name, nsg_id
"@
[array]$NSGsWithDenyAllRule = @(Search-AzGraph -Query $Query -UseTenantScope)
# we also need all NSG IDs
$Query = @"
resources
| where type == "microsoft.network/networksecuritygroups"
| project name, nsg_id = id
"@
#
[array]$NSGs = Search-AzGraph -Query $Query -UseTenantScope
# and finally we can compare and find which NSGs does not have such a rule
Compare-Object -ReferenceObject $NSGs -DifferenceObject $NSGsWithDenyAllRule -Property nsg_id -PassThru | Select-Object name, @{name='id';expression={$_.nsg_id}}


# Open Management Ports

Below query identifies NSG rules that expose management ports to the Internet. It will not find obscure rules like allowing *3380-3390*.

Expose your Virtual Machines using Azure Bastion or other similar service. Avoid exposing VMs directly to the Internet.

In [ ]:
# check for open management ports
$Query = @"
resources
| where type == "microsoft.network/networksecuritygroups"
| extend p = todynamic(properties.securityRules)
| mvexpand p
| extend destinationPortRange = p.properties.destinationPortRange
| extend destinationPortRanges = p.properties.destinationPortRanges
| extend sourceAddressPrefix = p.properties.sourceAddressPrefix
| extend access = p.properties.access
| extend direction = p.properties.direction
| where access == "Allow" and direction == "Inbound" 
| where destinationPortRange contains "22" or destinationPortRange contains "3389" or destinationPortRanges has "22" or destinationPortRanges has "3389"
| where sourceAddressPrefix == '*' or sourceAddressPrefix == 'Internet'
| extend port = strcat(destinationPortRange, iff(array_length( destinationPortRanges) == 0, '', destinationPortRanges))
| extend nsg_id = substring(p.id, 0, strlen(p.id) - strlen(strcat('/securityRules/',p.name)))
| join kind=inner (
	resources
	| where type == "microsoft.network/networksecuritygroups"
	| project name, nsg_id = id
) on nsg_id
| project ["NSG rule name"] = p.name, ["Exposed port"] = port, ["NSG name"] = name, id = nsg_id
"@
(Search-AzGraph -Query $Query -UseTenantScope).Data | Select-Object -ExcludeProperty ResourceId

# NSG Flow Logs

Network Security Groups should have flow logs enabled.

A more recent alternative (preview) is Virtual Network Flow Logs. This gives more insight into network traffic flows, but obviously does not tell you what an NSG blocked or allowed.

Best practice is to enforce NSG Flow Logs using Azure Policies.

Below will show Network Security Groups without flow logs enabled.

In [ ]:
# 
$Query = @"
resources
| where type =~ "microsoft.network/networkwatchers/flowlogs"
| where properties.enabled == true
| extend nsg_id = tostring(properties.targetResourceId)
| extend flowAnalytics = properties.flowAnalyticsConfiguration.networkWatcherFlowAnalyticsConfiguration.enabled
| project nsg_id, flowAnalytics
| join(
    resources
    | where type =~ "microsoft.network/networksecuritygroups"
    | mvexpand subnet = parse_json(properties.subnets)
    | project nsg_id = tostring(id), name, subnet
    ) on nsg_id
| project Name = name, ["Subnet"] = subnet["id"], ['Flow analytics enabled'] = flowAnalytics, nsg_id
"@
[array]$NSGsWithFlowLogs = (Search-AzGraph -Query $Query -UseTenantScope).Data | Select-Object -ExcludeProperty ResourceId

# we also need all NSG IDs
$Query = @"
resources
| where type == "microsoft.network/networksecuritygroups"
| project name, nsg_id = id
"@
#
[array]$NSGs = Search-AzGraph -Query $Query -UseTenantScope
# and finally we can compare and find which NSGs does not have such a rule
Compare-Object -ReferenceObject $NSGs -DifferenceObject $NSGsWithFlowLogs -Property nsg_id -PassThru | Select-Object name, @{name='id';expression={$_.nsg_id}}

# Network ACLs

Services like Storage Account and Key Vaults has a builtin *firewall* that allows for some basic network restrictions. Note that SQL Server firewall rules are *not* included here as they are (still) not accessable in Azure Resource Graph.

Below maps out the default action where *Allow* equates to *Enabled from all networks* and *Deny* is *Enabled from selected virtual networks and IP addresses* or *Disabled*

In almost all cases the default action should be set to *Deny*

In [ ]:
# 
$Query = @"
resources
| where isnotempty(properties.networkAcls)
| extend acl_default = tostring(properties.networkAcls.defaultAction), iif(isnotempty(properties.ipRangeFilter) and properties.ipRangeFilter contains "0.0.0.0", "Azure", "")
| where isnotempty(acl_default)
| sort by acl_default, type
| project Name = name, ["Service Type"] = type, ["ACL Default"] = acl_default

"@
(Search-AzGraph -Query $Query -UseTenantScope).Data | Select-Object -ExcludeProperty ResourceId

# Azure SQL Network ACLs

Checking Azure SQL firewall rules that allow excessive access (anything greater than an addresses range of more than 4 IP addresses).

`AllowAllWindowsAzureIps` is the checkmark that *Allows Azure services...* (This option configures the firewall to allow connections from IP addresses allocated to any Azure service or asset, including connections from the subscriptions of other customers.)

In [ ]:
# 
$Query = @"
resources
| where type == "microsoft.sql/servers"
| project name, resourceGroup
"@
[array]$SqlServers = (Search-AzGraph -Query $Query -UseTenantScope).Data | Select-Object -ExcludeProperty ResourceId

# Collect all SQL Server firewall rules
$SqlServerFirewallRules = $SqlServers | ForEach-Object {
    Get-AzSqlServerFirewallRule -ServerName $_.name -ResourceGroupName $_.resourceGroup
}
# Measure the number of IP addresses in each range
$null = $SqlServerFirewallRules | ForEach-Object {
    $Rule = $_
    $Rule | Add-Member -MemberType NoteProperty -Name IpAddressCount -Value (Measure-IpAddressCount -StartIpAddress $Rule.StartIpAddress -EndIpAddress $Rule.EndIpAddress)
}
# Filter to show only rules with excessive access
$SqlServerFirewallRules | Where-Object {$_.FirewallRuleName -eq 'AllowAllWindowsAzureIps' -or $_.IpAddressCount -gt 4} | Select-Object -Property ServerName, ResourceGroupName, FirewallRuleName | Format-Table -AutoSize